#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
import pandas as pd
from tqdm import tqdm
from warnings import warn
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pickle

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

## Huangrui's Dataset

In [2]:
default_mappings = {
    'label_maps': [{0: 'Did recid.', 1.0: 'No recid.'}],
    'protected_attribute_maps': [{0.0: 'Male', 1.0: 'Female'},
                                 {1.0: 'Caucasian', 0.0: 'Not Caucasian'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class CompasDataset(StandardDataset):
    """ProPublica COMPAS Dataset.

    See :file:`aif360/data/raw/compas/README.md`.
    """

    def __init__(self, label_name='Y', favorable_classes=[1],
                 protected_attribute_names=['sex'],
                 privileged_classes=[[1]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[],
                 features_to_drop=[], na_values=[],
                 custom_preprocessing=None,
                 metadata=default_mappings,
                 path='./Huangrui/recidivism/recidivism_test1.csv'):

    
        df = pd.read_csv(path,index_col=False)
        df.rename(columns={'Probationerssex_Female': 'sex'}, inplace=True)
        df.drop(["Probationerssex_Male","Probationerssex_Notascertained"], axis=1, inplace=True)
        numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
        code_continuous(df,numericals, 5)
        #flip the Y lable to 0: recid, 1: no recid
        df["Y"] = [1 if x == 0 else 0 for x in df["Y"]]
        
        super(CompasDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)


#### Load dataset and specify options

In [3]:
## import dataset
dataset_used = "compas" 

privileged_groups = [{'sex': 1}] #Females
unprivileged_groups = [{'sex': 0}]
        
# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

In [4]:
experiments_info = {}
budget = 0.01
for K in range(1, 6):
    dataset_orig_train= CompasDataset(path="./Huangrui/recidivism/recidivism_train{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    dataset_orig_test= CompasDataset(path="./Huangrui/recidivism/recidivism_test{}.csv".format(K),protected_attribute_names=['sex'],
                privileged_classes=[[1]])
    #only use the budget% of the training data
    # Get the dataset and split into train and test
    if budget == 0.01:
        #for budget = 0.01 use seed, we need to shuffle the data
        dataset_orig_train,_ = dataset_orig_train.split([budget ], shuffle=True,seed = 43)
    else:
        dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

    # Lasso linear classifier and predictions
    X_train = dataset_orig_train.features
    y_train = dataset_orig_train.labels.ravel()
    lmod = pickle.load(open('experiments/recidivism'+str(K)+'_sex_bmodel.pkl','rb'))["clf"]
   
    y_train_pred = lmod.predict(X_train)

    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    sigmoid = lambda x: 1 / (1 + np.exp(2-4*x))
    #beacuse we flip the y, we also need to flip the score
    dataset_orig_train_pred.scores = 1- sigmoid(y_train_pred).reshape(-1,1)

    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    X_test = dataset_orig_test_pred.features
    y_test = dataset_orig_test_pred.labels
    #beacuse we flip the y, we also need to flip the score
    dataset_orig_test_pred.scores = 1- sigmoid(lmod.predict(X_test)).reshape(-1,1)

    #### Best threshold for classification only (no fairness)
    num_thresh = 100
    ba_arr = np.zeros(num_thresh)
    class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
    for idx, class_thresh in enumerate(class_thresh_arr):
        
        fav_inds = dataset_orig_train_pred.scores > class_thresh
        dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
        dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
        
        classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                                dataset_orig_train_pred, 
                                                unprivileged_groups=unprivileged_groups,
                                                privileged_groups=privileged_groups)
        
        ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                        +classified_metric_orig_train.true_negative_rate())

    best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
    best_class_thresh = class_thresh_arr[best_ind]

    # print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
    # print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)
    #### Estimate optimal parameters for the ROC method
    ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                    privileged_groups=privileged_groups, 
                                    low_class_thresh=0.01, high_class_thresh=0.99,
                                    num_class_thresh=100, num_ROC_margin=50,
                                    metric_name=metric_name,
                                    metric_ub=metric_ub, metric_lb=metric_lb)
    ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)
    # print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
    # print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

    # Metrics for the train set
    fav_inds = dataset_orig_train_pred.scores > best_class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

    # display(Markdown("#### train set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                    unprivileged_groups, privileged_groups)

    # Transform the validation set
    dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

    # display(Markdown("#### train set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                    unprivileged_groups, privileged_groups)
    # Testing: Check if the metric optimized has not become worse
    assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

    # Metrics for the test set
    fav_inds = dataset_orig_test_pred.scores > best_class_thresh
    dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
    dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

    metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                    unprivileged_groups, privileged_groups)
    # Metrics for the transformed test set
    dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

    # display(Markdown("#### Test set"))
    # display(Markdown("##### Transformed predictions - With fairness constraints"))
    metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                    unprivileged_groups, privileged_groups)

    print("K = {}, budget = {}".format(K, budget))
    print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
    print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))
    experiments_info["K = {}, budget = {}".format(K, budget)] = {"Error": np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels), "Equal opportunity difference": metric_test_aft["Equal opportunity difference"]}

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8077
Statistical parity difference = -0.1151
Disparate impact = 0.7929
Average odds difference = -0.1206
Equal opportunity difference = 0.0333
Theil index = 0.2158
Balanced accuracy = 0.8077
Statistical parity difference = -0.1151
Disparate impact = 0.7929
Average odds difference = -0.1206
Equal opportunity difference = 0.0333
Theil index = 0.2158
Balanced accuracy = 0.7082
Statistical parity difference = -0.1945
Disparate impact = 0.7022
Average odds difference = -0.1360
Equal opportunity difference = -0.1223
Theil index = 0.2762
Balanced accuracy = 0.7082
Statistical parity difference = -0.1945
Disparate impact = 0.7022
Average odds difference = -0.1360
Equal opportunity difference = -0.1223
Theil index = 0.2762
K = 1, budget = 0.01
The Error for the test dataset is 0.3052
The Equal opportunity difference for the test dataset is -0.1223


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.7639
Statistical parity difference = -0.3302
Disparate impact = 0.5597
Average odds difference = -0.2720
Equal opportunity difference = -0.1617
Theil index = 0.2509
Balanced accuracy = 0.7734
Statistical parity difference = -0.0617
Disparate impact = 0.9074
Average odds difference = 0.1620
Equal opportunity difference = 0.0298
Theil index = 0.1491
Balanced accuracy = 0.7438
Statistical parity difference = -0.2321
Disparate impact = 0.6961
Average odds difference = -0.1610
Equal opportunity difference = -0.1685
Theil index = 0.1971
Balanced accuracy = 0.7229
Statistical parity difference = -0.0198
Disparate impact = 0.9712
Average odds difference = 0.0559
Equal opportunity difference = 0.0295
Theil index = 0.1460
K = 2, budget = 0.01
The Error for the test dataset is 0.2503
The Equal opportunity difference for the test dataset is 0.02949


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8780
Statistical parity difference = -0.3574
Disparate impact = 0.5916
Average odds difference = -0.0725
Equal opportunity difference = -0.2075
Theil index = 0.1348
Balanced accuracy = 0.8576
Statistical parity difference = -0.1500
Disparate impact = 0.8000
Average odds difference = 0.0835
Equal opportunity difference = 0.0108
Theil index = 0.1098
Balanced accuracy = 0.6771
Statistical parity difference = -0.1490
Disparate impact = 0.7570
Average odds difference = -0.1156
Equal opportunity difference = -0.0733
Theil index = 0.3030
Balanced accuracy = 0.6567
Statistical parity difference = 0.2671
Disparate impact = 2.0361
Average odds difference = 0.2738
Equal opportunity difference = 0.3701
Theil index = 0.3114
K = 3, budget = 0.01
The Error for the test dataset is 0.3529
The Equal opportunity difference for the test dataset is 0.3701


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.7930
Statistical parity difference = -0.2976
Disparate impact = 0.6693
Average odds difference = -0.4224
Equal opportunity difference = -0.0448
Theil index = 0.1436
Balanced accuracy = 0.8087
Statistical parity difference = -0.1976
Disparate impact = 0.7530
Average odds difference = -0.1724
Equal opportunity difference = -0.0448
Theil index = 0.1406
Balanced accuracy = 0.7154
Statistical parity difference = -0.1329
Disparate impact = 0.8145
Average odds difference = -0.1067
Equal opportunity difference = -0.0282
Theil index = 0.1943
Balanced accuracy = 0.7132
Statistical parity difference = -0.0396
Disparate impact = 0.9366
Average odds difference = -0.0082
Equal opportunity difference = 0.0592
Theil index = 0.2034
K = 4, budget = 0.01
The Error for the test dataset is 0.2774
The Equal opportunity difference for the test dataset is 0.05923


C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Balanced accuracy = 0.8208
Statistical parity difference = -0.2875
Disparate impact = 0.6646
Average odds difference = -0.2906
Equal opportunity difference = -0.0758
Theil index = 0.1284
Balanced accuracy = 0.8270
Statistical parity difference = -0.1447
Disparate impact = 0.7975
Average odds difference = -0.0784
Equal opportunity difference = 0.0152
Theil index = 0.1374
Balanced accuracy = 0.7323
Statistical parity difference = -0.1428
Disparate impact = 0.8227
Average odds difference = -0.1108
Equal opportunity difference = -0.0809
Theil index = 0.1334
Balanced accuracy = 0.7337
Statistical parity difference = 0.0278
Disparate impact = 1.0435
Average odds difference = 0.0835
Equal opportunity difference = 0.0515
Theil index = 0.1433
K = 5, budget = 0.01
The Error for the test dataset is 0.2414
The Equal opportunity difference for the test dataset is 0.05155


In [5]:
experiments_info

{'K = 1, budget = 0.01': {'Error': 0.3051502145922747,
  'Equal opportunity difference': -0.12232242026762574},
 'K = 2, budget = 0.01': {'Error': 0.25032202662086733,
  'Equal opportunity difference': 0.0294946017837584},
 'K = 3, budget = 0.01': {'Error': 0.35294117647058826,
  'Equal opportunity difference': 0.37007649363067147},
 'K = 4, budget = 0.01': {'Error': 0.2773722627737226,
  'Equal opportunity difference': 0.059225850857650086},
 'K = 5, budget = 0.01': {'Error': 0.24140893470790378,
  'Equal opportunity difference': 0.0515497553017944}}